In [1]:
import pandas as pd
import numpy as np
import folium
from geopy.distance import geodesic
import joblib  # to load the trained price model


## 1. Data Input

In [2]:
df = pd.read_csv("../../data/processed/cleaned_resale_data.csv")

# Using simulated data for demonstration purposes first
df["predicted_price"] = df["resale_price"] * np.random.uniform(0.95, 1.05, len(df))  # Simulated


## 2. Budget Input

In [5]:
budget_temp = 500000

## 3. Filter Affordable choices

In [6]:
def get_affordable_flats(predicted_price, resale_df, top_n=50):
    """
    Filters and returns flats within the user's predicted price range.
    """
    affordable = resale_df[resale_df["predicted_price"] <= predicted_price].copy()
    print(f"✅ Found {len(affordable)} flats under budget ${predicted_price:,.0f}")
    return affordable.head(top_n)

affordable_flats = get_affordable_flats(budget_temp, df)
affordable_flats.head()


✅ Found 110253 flats under budget $500,000


,time,address,storey_avg,floor_area_sqm,flat_type_encoded,flat_model,remaining_lease_months,resale_price,predicted_price
0,0,406 ANG MO KIO AVE 10,11,44.0,2,Improved,736,232000.0,240697.008264
1,0,108 ANG MO KIO AVE 4,2,67.0,3,New Generation,727,250000.0,254929.095825
2,0,602 ANG MO KIO AVE 5,2,67.0,3,New Generation,749,262000.0,273713.770015
3,0,465 ANG MO KIO AVE 10,5,68.0,3,New Generation,745,265000.0,265822.839231
4,0,601 ANG MO KIO AVE 5,2,67.0,3,New Generation,749,265000.0,270031.498187


## Getting coordinates

In [ ]:
# Using Simulated Data for Housing Locations, MRT and Schools
# The OneMap API is too slow in this environment, so we will simulate the data instead.
SG_LAT_MIN, SG_LAT_MAX = 1.22, 1.47
SG_LNG_MIN, SG_LNG_MAX = 103.6, 104.0

np.random.seed(42)
affordable_flats["lat"] = np.random.uniform(SG_LAT_MIN, SG_LAT_MAX, len(affordable_flats))
affordable_flats["lng"] = np.random.uniform(SG_LNG_MIN, SG_LNG_MAX, len(affordable_flats))

mrt_df = pd.DataFrame({
    "station": [f"MRT_{i}" for i in range(30)],
    "lat": np.random.uniform(SG_LAT_MIN, SG_LAT_MAX, 30),
    "lng": np.random.uniform(SG_LNG_MIN, SG_LNG_MAX, 30)
})


schools_df = pd.DataFrame({
    "school": [f"School_{i}" for i in range(50)],
    "lat": np.random.uniform(SG_LAT_MIN, SG_LAT_MAX, 50),
    "lng": np.random.uniform(SG_LNG_MIN, SG_LNG_MAX, 50)
})

amenities_df = pd.DataFrame({
    "amenity": [f"Amenity_{i}" for i in range(20)],
    "lat": np.random.uniform(SG_LAT_MIN, SG_LAT_MAX, 20),
    "lng": np.random.uniform(SG_LNG_MIN, SG_LNG_MAX, 20)
})





NameError: name 'df' is not defined

## Add Scoring Function

In [ ]:
from geopy.distance import geodesic

def score_location(lat, lng, mrt_df, schools_df, amenities_df):
    def count_within(df, radius_km):
        return df.apply(lambda row: geodesic((lat, lng), (row["lat"], row["lng"])).km <= radius_km, axis=1).sum()

    score = 0
    score += min(count_within(mrt_df, 0.8), 3) * 2        # MRT: up to 6 pts
    score += min(count_within(schools_df, 1.0), 3) * 1.5  # Schools: up to 4.5 pts
    score += min(count_within(amenities_df, 1.0), 2) * 1  # Amenities: up to 2 pts
    
    return round(min(score, 10), 2)

affordable_flats["score"] = affordable_flats.apply(
    lambda row: score_location(row["lat"], row["lng"], mrt_df, schools_df, amenities_df), axis=1
)

top_flats = affordable_flats.sort_values(by="score", ascending=False).head(20)
top_flats[["address", "predicted_price", "score", "lat", "lng"]]


In [20]:
import requests
import time
import pandas as pd

def get_coordinates(address):
    """Fetch coordinates (lat, lng) for a given address using OneMap API."""
    url = "https://www.onemap.gov.sg/api/common/elastic/search"
    params = {
        "searchVal": address,
        "returnGeom": "Y",
        "getAddrDetails": "Y",
        "pageNum": 1
    }
    try:
        response = requests.get(url, params=params, timeout=5)
        results = response.json().get("results", [])
        if results:
            lat = float(results[0]["LATITUDE"])
            lng = float(results[0]["LONGITUDE"])
            return lat, lng
    except Exception as e:
        print(f"❌ Error fetching {address}: {e}")
    return None, None
